In [14]:
!wget --no-check-certificate \
 https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.zip \
 -O /tmp/glove.zip

--2022-06-30 06:38:07--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.98.128, 142.250.97.128, 74.125.196.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.98.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 112372936 (107M) [application/zip]
Saving to: ‘/tmp/glove.zip’

/tmp/glove.zip      100%[===================>] 107.17M   142MB/s    in 0.8s    

2022-06-30 06:38:08 (142 MB/s) - ‘/tmp/glove.zip’ saved [112372936/112372936]



In [15]:
import os
import zipfile
local_zip = '/tmp/glove.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/glove')
zip_ref.close()

In [17]:
import numpy as np
glove_embeddings = dict()
f = open('/tmp/glove/glove.twitter.27B.25d.txt')
for line in f:
 values = line.split()
 word = values[0]
 coefs = np.asarray(values[1:], dtype='float32')
 glove_embeddings[word] = coefs
f.close()


In [18]:
glove_embeddings

{'<user>': array([ 0.62415 ,  0.62476 , -0.082335,  0.20101 , -0.13741 , -0.11431 ,
         0.77909 ,  2.6356  , -0.46351 ,  0.57465 , -0.024888, -0.015466,
        -2.9696  , -0.49876 ,  0.095034, -0.94879 , -0.017336, -0.86349 ,
        -1.3348  ,  0.046811,  0.36999 , -0.57663 , -0.48469 ,  0.40078 ,
         0.75345 ], dtype=float32),
 '.': array([ 0.69586 , -1.1469  , -0.41797 , -0.022311, -0.023801,  0.82358 ,
         1.2228  ,  1.741   , -0.90979 ,  1.3725  ,  0.1153  , -0.63906 ,
        -3.2252  ,  0.61269 ,  0.33544 , -0.57058 , -0.50861 , -0.16575 ,
        -0.98153 , -0.8213  ,  0.24333 , -0.14482 , -0.67877 ,  0.7061  ,
         0.40833 ], dtype=float32),
 ':': array([ 1.1242  ,  0.054519, -0.037362,  0.10046 ,  0.11923 , -0.30009 ,
         1.0938  ,  2.537   , -0.072802,  1.0491  ,  1.0931  ,  0.066084,
        -2.7036  , -0.14391 , -0.22031 , -0.99347 , -0.65072 , -0.030948,
        -1.0817  , -0.64701 ,  0.32341 , -0.41612 , -0.5268  , -0.047166,
         0.71549 ], 

In [19]:
glove_embeddings['start']

array([ 0.12451 ,  1.3797  ,  0.54486 , -0.90407 , -0.11113 ,  1.1818  ,
        1.1898  , -0.5814  ,  0.35164 ,  0.20501 , -0.56733 ,  0.67764 ,
       -4.277   ,  0.035894,  0.078868,  0.47072 , -0.37612 , -0.68964 ,
       -1.1399  , -1.3196  , -0.62336 , -0.095528,  0.013147,  0.33991 ,
       -0.088342], dtype=float32)

In [21]:
import json
import string
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words('english')
table = str.maketrans('','',string.punctuation)
with open("/Sarcasm_Headlines_Dataset.json", 'r') as f:
  datastore = [json.loads(line) for line in f]
sentences = []
labels = []
urls = []
for item in datastore:
  sentence = item['headline'].lower()
  sentence = sentence.replace(",", " , ")
  sentence = sentence.replace(".", " . ")
  sentence = sentence.replace("-", " - ")
  sentence = sentence.replace("/", " / ")
  soup = BeautifulSoup(sentence)
  sentence = soup.get_text()
  words = sentence.split()
  filtered_sentence = ""
  for word in words:
   word = word.translate(table)
   if word not in stopwords:
     filtered_sentence = filtered_sentence + word + " "
  sentences.append(filtered_sentence)
  labels.append(item['is_sarcastic'])
  urls.append(item['article_link'])
  training_size = 25000
tr_sentences = sentences[0:training_size]
tr_labels = labels[0:training_size]
t_sentences = sentences[training_size:]
t_labels = labels[training_size:]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = 10000,oov_token = '<OOV>')

In [24]:
tokenizer.fit_on_texts(sentences)
tr_sequence = tokenizer.texts_to_sequences(tr_sentences)
from tensorflow.keras.preprocessing.sequence import pad_sequences
tr_padded = pad_sequences(tr_sequence,padding='post',maxlen=20)

In [28]:
embedding_matrix = np.zeros((10000, 25))
for word, index in tokenizer.word_index.items():
 if index > 10000 - 1:
  break
 else:
   embedding_vector = glove_embeddings.get(word)
   if embedding_vector is not None:
     embedding_matrix[index] = embedding_vector

In [32]:
len(embedding_matrix)
import tensorflow as tf

In [33]:
model = tf.keras.Sequential([
 tf.keras.layers.Embedding(10000, 25,
 weights=[embedding_matrix], trainable=False),
 tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(25,
 return_sequences=True)),
 tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(25)),
 tf.keras.layers.Dense(24, activation='relu'),
 tf.keras.layers.Dense(1, activation='sigmoid')
])


In [34]:
adam = tf.keras.optimizers.Adam(learning_rate=0.00001, beta_1=0.9, beta_2=0.999,
amsgrad=False)
model.compile(loss='binary_crossentropy',optimizer=adam, metrics=['accuracy'])


In [ ]:
model.fit(np.array(tr_padded),np.array(tr_labels),epochs = 30)

Epoch 1/30
782/782 [==============================] - 22s 28ms/step - loss: 0.6118 - accuracy: 0.6674
Epoch 2/30
782/782 [==============================] - 22s 28ms/step - loss: 0.6055 - accuracy: 0.6718
Epoch 3/30
782/782 [==============================] - 22s 28ms/step - loss: 0.6012 - accuracy: 0.6751
Epoch 4/30
782/782 [==============================] - 22s 28ms/step - loss: 0.5979 - accuracy: 0.6804
Epoch 5/30
782/782 [==============================] - 23s 30ms/step - loss: 0.5955 - accuracy: 0.6827
Epoch 6/30
782/782 [==============================] - 22s 28ms/step - loss: 0.5930 - accuracy: 0.6854
Epoch 7/30
782/782 [==============================] - 22s 28ms/step - loss: 0.5909 - accuracy: 0.6887
Epoch 8/30
782/782 [==============================] - 22s 28ms/step - loss: 0.5887 - accuracy: 0.6904
Epoch 9/30
782/782 [==============================] - 22s 28ms/step - loss: 0.5868 - accuracy: 0.6930
Epoch 10/30
782/782 [==============================] - 22s 28ms/step - loss: 0.584